In [1]:
%reset -f
from django_for_jupyter import init_django
init_django()

/home/javier/hd1/Codigo/JAVIER/test1/comparaprecios-1


In [2]:
from precios.models import (Marcas, Articulos, Vendedores, Unifica, Settings)
from tabulate import tabulate
import pandas as pd
from precios.pi_rules import (
    intenta_marca,
    create_PD_From
)   

In [7]:
# Unifica.objects.filter(automatico=True).delete() 

# Vendedores.objects.all().delete()
# Articulos.objects.all().delete()

(47971, {'precios.Articulos': 47971})

In [4]:
# marca = Marcas.objects.filter(nombre="loncoleche").get()
# intenta_marca(marca, True)

articles_from_all = Vendedores.objects.select_related('articulo','vendidoen')
# articles_from_all = articles_from_all.filter(articulo__marca=marca)
articles_from_all = articles_from_all.exclude(vendidoen__precio=0)
articles_from_all = articles_from_all.exclude(vendidoen__error404=True)
articles_from_all = articles_from_all.distinct()


In [5]:
df_articulos = pd.DataFrame(list(articles_from_all.values('articulo__pk','articulo__marca','articulo__nombre','articulo__talla',)))
df_articulos['quienesvenden']   = {}

for cuenta, row in df_articulos.iterrows():
    art = Articulos.objects.get(pk=row['articulo__pk'])
    df_articulos.at[cuenta,'quienesvenden']         = art.quienesvenden()

In [6]:
def has_duplicates(lst):
    return len(lst) != len(set(lst))


# Aplicar la función para encontrar listas con elementos duplicados
df_articulos['has_duplicates'] = df_articulos['quienesvenden'].apply(has_duplicates)

# Filtrar las filas que tienen listas con elementos duplicados
df_filtered = df_articulos[df_articulos['has_duplicates']]

# Mostrar las filas filtradas
print(tabulate(df_filtered, headers = 'keys', tablefmt = 'psql'))


+-------+----------------+-------------------+-------------------------------------------------------------------------+-------------------+------------------------------------------------------+------------------+
|       |   articulo__pk |   articulo__marca | articulo__nombre                                                        | articulo__talla   | quienesvenden                                        | has_duplicates   |
|-------+----------------+-------------------+-------------------------------------------------------------------------+-------------------+------------------------------------------------------+------------------|
|    27 |       36613464 |           1115022 | promo roble del sur 1 l + coca cola original 1 5 l + hielo              |                   | [56, 56, 56, 68]                                     | True             |
|    28 |       36613329 |           1115022 | promo roble del sur 1 l + coca cola original 1 5 l + hielo              |                   |